In [1]:
import requests
import numpy as np
import pandas as pd
import itertools
import multiprocessing as mp
import datetime
import time
import os
pd.plotting.register_matplotlib_converters()
#api_key = '{my_api_key}'
url = 'https://api.polygon.io/v2/ticks/stocks/trades/{symbol}/{date}?timestamp={timestamp}&limit={limit}&apiKey={api_key}'

# Get trade data from polygon.io

In [4]:
def get_trades(date):
    params = {
    'symbol': 'SPY',
    'date': date,
    'timestamp': '0',
    'limit': 50000,
    'api_key': api_key
    }
    pdata = []
    ct           = params['limit']
    it           = 1
    while ct >= params['limit']:
        response = requests.get(url.format(**params)).json()
        if 'results' in response:
            ct                  = response['results_count']
            pdata              += response['results']
            params['timestamp'] = response['results'][-1]['t']
        else:
            ct = 0
        #print('\r{} | {}...'.format(date, it), end='')
        it += 1
    return pdata

def get_historic_trades(date_start, date_end):
    #get data and write to csv
    dates = [str(x)[:10] for x in pd.date_range(date_start, date_end, freq='D')]
    sd    = 0
    fnum  = 0
    while sd < len(dates):
        with mp.Pool(8) as pool:
            pricing_data = list(itertools.chain.from_iterable(pool.map(get_trades, dates[sd:(sd+100)])))
        print('Exporting {} to {}...'.format(dates[sd], dates[sd+99]))
        spydf = pd.DataFrame(pricing_data)
        spydf.to_csv('SPY{}.csv'.format(fnum), index = False)
        del spydf
        del pricing_data
        sd   += 100
        fnum +=1
    return
    
def export(num):
    #read in trades, export each year to separate file
    print('Reading in SPY{}'.format(num))
    df = pd.read_csv('SPY{}.csv'.format(num), index_col = ['t'], engine = 'c')
    print('\tParsing dates')
    df.index = pd.to_datetime(df.index, unit='ns')
    print('\tParsing year')
    print('\tExporting')
    for k, g in df.groupby(df.index.year):
        print('\t\t{}'.format(k))
        g.to_csv('SPY_{}_{}.csv'.format(k, num), index=False)
    return